In [8]:
import altair as alt
from vega_datasets import data

source = data.cars()

source


,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA
...,...,...,...,...,...,...,...,...,...
401,ford mustang gl,27.0,4,140.0,86.0,2790,15.6,1982-01-01,USA
402,vw pickup,44.0,4,97.0,52.0,2130,24.6,1982-01-01,Europe
403,dodge rampage,32.0,4,135.0,84.0,2295,11.6,1982-01-01,USA
404,ford ranger,28.0,4,120.0,79.0,2625,18.6,1982-01-01,USA


In [28]:
# Brush for selection
brush = alt.selection_single(on="mouseover")

# Scatter Plot
points = alt.Chart(source).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey'))
).add_selection(brush)

# Base chart for data tables
ranked_text = alt.Chart(source).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
)

In [30]:
# Data Tables
horsepower = ranked_text.encode(text='Horsepower:N').properties(title='Horsepower')
mpg = ranked_text.encode(text='Miles_per_Gallon:N').properties(title='MPG')
origin = ranked_text.encode(text='Origin:N').properties(title='Origin')
text = alt.hconcat(horsepower, mpg, origin) # Combine data tables

# Build chart
alt.hconcat(
    points,
    text
)

alt.HConcatChart(...)

In [48]:
from dataclasses import fields
import streamlit as st
import pandas as pd
import altair as alt
from re import U
import urllib.request
import ssl
from scipy import stats
from vega_datasets import data
import numpy as np


# sc_df = pd.read_csv("social_capital_zip.csv")
# sc_df.reset_index(inplace=True)
# sc_df.zip = sc_df.zip.astype(str)

# zip_and_coords = pd.read_table("US Zip Codes from 2013 Government Data")
# zip_and_coords.rename(columns={"ZIP,LAT,LNG": "zip"}, inplace=True)
# zip_and_coords = zip_and_coords['zip'].str.split(',', expand=True)
# zip_and_coords.rename(columns={0: "zip", 1: "lat", 2: "lng"}, inplace=True)

# Merge the two dataframes
# sc_df = pd.merge(sc_df, zip_and_coords, on='zip')
#sc_df.to_csv("social_capital_zip_coords.csv")

## Add State, County, and City to the df from another csv file.



def load_data():
    return pd.read_csv("data/social_capital_zip_coords.csv")

df = load_data()
df = df.head(100)

df

,Unnamed: 0,index,zip,county,num_below_p50,pop2018,ec_zip,ec_se_zip,nbhd_ec_zip,ec_grp_mem_zip,...,bias_grp_mem_zip,bias_grp_mem_high_zip,nbhd_bias_zip,nbhd_bias_high_zip,clustering_zip,support_ratio_zip,volunteering_rate_zip,civic_organizations_zip,lat,lng
0,0,1931,10001,36061.0,909.618268,22924,0.83768,0.01741,NaN,1.10926,...,0.03933,-0.07902,NaN,NaN,0.083200,0.554760,0.052950,0.033670,40.750633,-73.997177
1,1,1932,10002,36061.0,14952.651726,74993,0.86278,0.01468,0.96910,1.01334,...,0.19741,-0.06841,0.38141,-0.05916,0.085210,0.809430,0.060860,0.009230,40.715775,-73.986212
2,2,1933,10003,36061.0,1457.991344,54682,0.69477,0.01288,0.96008,1.06211,...,0.04605,-0.07615,0.28338,-0.43447,0.084430,0.670310,0.049320,0.007490,40.731829,-73.989181
3,3,1934,10004,36061.0,34.181794,3028,0.96205,0.04533,NaN,1.24840,...,-0.02065,-0.06848,NaN,NaN,0.092780,0.317610,0.054680,0.070740,40.688630,-74.018244
4,4,1935,10005,36061.0,7.864282,8831,1.25535,0.03834,NaN,1.41811,...,-0.03400,-0.07450,NaN,NaN,0.089970,0.321110,0.052540,0.000060,40.706027,-74.008835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,2026,10522,36119.0,535.484802,11027,1.08438,0.03498,NaN,1.31833,...,0.00828,-0.08491,NaN,NaN,0.092170,0.934710,0.099610,0.020030,41.010310,-73.863746
96,96,2027,10523,36119.0,557.605919,9077,0.86637,0.02323,1.18919,1.13566,...,0.09117,-0.05434,0.13260,-0.10730,0.097530,0.919370,0.037760,0.011700,41.059341,-73.819457
97,97,2028,10524,36079.0,146.386235,4227,0.96892,0.05339,NaN,NaN,...,NaN,NaN,NaN,NaN,0.094470,0.877600,0.095080,0.037570,41.375325,-73.926217
98,98,2029,10526,36119.0,57.551559,1572,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.093854,0.643705,0.073805,0.014669,41.288102,-73.668835


In [60]:
# use altair to create map chart
# altair supports tooltip on datapoints, but currently doesn't support to zoom in/out a map chart
# reference: https://altair-viz.github.io/altair-tutorial/notebooks/09-Geographic-plots.html

# background map
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=600,
    height=400
)

dimension_dic = {
    "Economic Connectedness": "ec_zip",
    "Cohesiveness": "clustering_zip", 
    "Civic Engagement":"volunteering_rate_zip"
    }
map_tooltip_value = "ec_zip"


# the data points on the map background
single_brush = alt.selection_single(on='click',fields=['zip'])
multi_brush = alt.selection_multi(fields=['zip'])
points = alt.Chart(df).mark_circle().encode(
    longitude = 'lng',
    latitude = 'lat',
    tooltip = ['zip', "ec_zip"],
    # color = alt.condition(single_brush, alt.value('salmon'), alt.value('lightgray'))
).properties(
    width = 200,
    height = 100
)


# bar = alt.Chart(df).mark_bar().encode(
#     x='zip',
#     y='ec_zip',
# ).transform_filter(single_brush).interactive()

# st.write((background+points)&bar)

zip_text = alt.Chart(df).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    single_brush
).transform_window(
    rank='rank(row_number)'
)
economic_connectedness = zip_text.encode(text='ec_zip').properties(title='Economic Connectedness')
# text = alt.hconcat(horsepower, mpg, origin) # Combine data tables

map_with_text = alt.hconcat(
    background+points)


points

alt.Chart(...)

In [ ]:
st.write(map_with_text&economic_connectedness)

#  st.write(str(single_brush))
test_brush = alt.selection_single()


zip_bar = alt.Chart(df).mark_bar().encode(
    x='zip',
    y='ec_zip',
    color=alt.condition(test_brush, alt.value('lightblue'), alt.value('lightgray'))
).add_selection(test_brush).interactive()

#st.write(zip_bar)

detail_bar = alt.Chart(df).mark_bar().encode(
    x='zip',
    y='num_below_p50'
).transform_filter(test_brush).interactive()